# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mnist
import tensorflow as tf
import keras

import warnings 

warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [12]:
#import the dataset 

from keras.datasets import mnist

#import various components for model building 
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

#import the backend
from keras import backend as K

In [5]:
from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 2s 0us/step


In [8]:
x_train

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

# Processing Data

In [9]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)


#convert to float32 for type consistency 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255

#print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#convert class vectors to binary class metrices
#so instead of one column with 10 values, create 10 binary columns 
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


# MLP

In [13]:
#start with a simple sequetial model

model = Sequential()

#add dense layers to create a fully connected MLP
#note that we specify an input shape for the first layer ONLY
#Relu is the activation function used 
model.add(Dense(64, activation='relu', input_shape=(784,)))
#dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
#end with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

#compile the model to put it all together

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s 18us/step - loss: 0.6512 - accuracy: 0.7701 - val_loss: 0.4636 - val_accuracy: 0.8350
Epoch 2/10
60000/60000 [==============================] - 1s 18us/step - loss: 0.4405 - accuracy: 0.8390 - val_loss: 0.4066 - val_accuracy: 0.8537
Epoch 3/10
60000/60000 [==============================] - 1s 16us/step - loss: 0.3960 - accuracy: 0.8564 - val_loss: 0.4063 - val_accuracy: 0.8561
Epoch 4/10
60000/60000 [==============================] - 1s 15us/step - loss: 0.3689 - accuracy: 0.8657 - val_loss: 0.4143 - val_accuracy: 0.8538
Epoch 5/10
60000/60000 [==============================] - 1s 16us/step - loss: 0.3511 - accuracy: 0.8726 - val_loss: 0.3788 - val_accuracy: 0.8664
Epoch 6/10
60000/60000 [==============================] - 1s 16us/step - loss: 0.3400 - accuracy: 0.8772 - val_loss: 0.3546 - val_accuracy: 0.8749
Epoch 7/10
60000/60000 [==============================] - 1s 16us/st

# CNN

In [15]:
#input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 10

#the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
    
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#convert class vectors to binary class matrices 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#building the model 
model = Sequential()
#first convolutional layer, note the specification of shape 
model.add(Conv2D(32, kernel_size=(3,3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 61s 1ms/step - loss: 0.2577 - accuracy: 0.9212 - val_loss: 0.0499 - val_accuracy: 0.9843
Epoch 2/10
60000/60000 [==============================] - 59s 978us/step - loss: 0.0863 - accuracy: 0.9749 - val_loss: 0.0395 - val_accuracy: 0.9859
Epoch 3/10
60000/60000 [==============================] - 56s 938us/step - loss: 0.0639 - accuracy: 0.9814 - val_loss: 0.0355 - val_accuracy: 0.9883
Epoch 4/10
60000/60000 [==============================] - 57s 942us/step - loss: 0.0541 - accuracy: 0.9836 - val_loss: 0.0318 - val_accuracy: 0.9884
Epoch 5/10
60000/60000 [==============================] - 57s 949us/step - loss: 0.0459 - accuracy: 0.9860 - val_loss: 0.0289 - val_accuracy: 0.9900
Epoch 6/10
60000/60000 [==============================] - 58s 964us/step - loss: 0.0422 - accuracy: 0.9871 - val_loss: 0.0288 - val_ac

# HRNN

In [16]:
#training parameters 
batch_size = 64
num_classes = 10
epochs = 3

#embedding dimensions 
row_hidden = 32
col_hidden = 32

#the data, shuffled and split between train and test sets 
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#reshape data to 4D for Hierarchical RNN
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#converts class vectors to binary class matrices 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

#4D input 
x = Input(shape=(row, col, pixel))

#encodes a row of pixels using TimeDistributed Wrapper
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

#encodes columns of encoded rows
encoded_columns = LSTM(col_hidden)(encoded_rows)

#final predictions and model 
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#training
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test)
         )
#evaluation 
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 122s 2ms/step - loss: 0.9877 - accuracy: 0.6641 - val_loss: 0.7358 - val_accuracy: 0.7481
Epoch 2/3
60000/60000 [==============================] - 121s 2ms/step - loss: 0.4307 - accuracy: 0.8637 - val_loss: 0.3000 - val_accuracy: 0.9052
Epoch 3/3
60000/60000 [==============================] - 121s 2ms/step - loss: 0.2706 - accuracy: 0.9163 - val_loss: 0.3349 - val_accuracy: 0.8906
Test loss: 0.3348750020325184
Test accuracy: 0.8906000256538391


CNN achieves the best accuracy. 